# Import everything


In [1]:
from typing import Tuple
from os.path import exists
import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from settings import cfg
from helper import loader
import export_result


# Load everything

In [2]:
config = cfg.TrainConfig()


Param size: 512.227MB
Buffer size: 0.032MB
Stats:
        | Number of not-punching: 34006
        | Number of punching: 6355


# Train

In [3]:
from sklearn.metrics import f1_score


def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    config.model.train()
    train_dataloader = config.get_dataloader(train_idx)
    total_loss_train = 0

    prediction_array = []
    label_array = []
    for image, label in tqdm(train_dataloader):
        image = image.to(config.device, dtype=torch.float)
        image = config.transforms(image)
        output = config.model(image)

        label = label.to(config.device, dtype=torch.uint8)
        batch_loss = config.criterion(output, label)
        total_loss_train += batch_loss.item()

        prediction = output.argmax(dim=1)

        prediction_array.append(prediction.cpu().numpy())
        label_array.append(label.cpu().numpy())

        config.optimizer.zero_grad()
        batch_loss.backward()
        config.optimizer.step()
        config.model.zero_grad()

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accuracy_train = (prediction_array == label_array).sum().item()
    f1_score_train = f1_score(label_array, prediction_array, average='macro')

    return (total_loss_train/train_idx.shape[0],
        total_accuracy_train/train_idx.shape[0],
        f1_score_train)


In [4]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    config.model.eval()
    judge_dataloader = config.get_dataloader(judge_idx)
    total_loss_judge = 0

    prediction_array = []
    label_array = []
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(config.device, dtype=torch.float)
            output = config.model(image)
            label = label.to(config.device, dtype=torch.uint8)

            batch_loss = config.criterion(output, label)
            total_loss_judge += batch_loss.item()

            prediction = output.argmax(dim=1)
            prediction_array.append(prediction.cpu().numpy())
            label_array.append(label.cpu().numpy())

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accuracy_judge = (prediction_array == label_array).sum().item()
    f1_score_judge = f1_score(label_array, prediction_array, average='macro')

    return (total_loss_judge/judge_idx.shape[0],
        total_accuracy_judge/judge_idx.shape[0],
        f1_score_judge)

In [5]:
min_judge_loss = float('inf')
last_submit = 0

for epoch, (train_idx, judge_idx) in enumerate(config.get_split()):
    print(f'''
Starting epoch {epoch+1}
    | Train size: {train_idx.shape[0]}
    | Judge size: {judge_idx.shape[0]}''', end="")
    avg_loss_train, avg_accu_train, f1_score_train = train(train_idx)
    avg_loss_judge, avg_accu_judge, f1_score_judge = judge(judge_idx)

    print(
        f'''
Epoch: {epoch+1} 
    | Train Loss: {avg_loss_train:.3f}   \t| Judge Loss: {avg_loss_judge:.3f}
    | Train Accuracy: {avg_accu_train:.3f} \t| Judge Accuracy: {avg_accu_judge:.3f}
    | Train F1 Score: {f1_score_train:.3f} \t| Judge F1 Score: {f1_score_judge:.3f}''', end="")

    if min_judge_loss > avg_loss_judge:
        config.save_checkpoint()
        print(f'''
Judge loss improved:
    | From: {min_judge_loss:.3f} | To: {avg_loss_judge:.3f}''', end="")
        min_judge_loss = avg_loss_judge
        under_min = 0
    else:
        under_min += 1
        if under_min > cfg.early_stop:
            print(f'''
Early stop. Not improved for {under_min} epochs.''', end="")
            config.load_best()
            print(f'''
Best model loaded.''', end="")
            under_min = 0

    if last_submit == 0:
        last_submit = f1_score_judge
    elif f1_score_judge - last_submit > 0.05:
        last_submit = f1_score_judge
        print(f'''
Submitting:
    | F1 Score: {f1_score_judge:.3f} | Last Submit: {last_submit:.3f}''')
        export_result.submit(config)
    else:
        print(f'''
Not submitting:
    | F1 Score: {f1_score_judge:.3f} | Last Submit: {last_submit:.3f}''', end="")
    print("\n____________________________________________")


Starting epoch 1
    | Train size: 36324
    | Judge size: 4037

  0%|          | 0/1136 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]


Epoch: 1 
    | Train Loss: 0.021   	| Judge Loss: 0.027
    | Train Accuracy: 0.587 	| Judge Accuracy: 0.834
    | Train F1 Score: 0.516 	| Judge F1 Score: 0.515
Judge loss improved:
    | From: inf | To: 0.027
____________________________________________

Starting epoch 2
    | Train size: 36325
    | Judge size: 4036

  0%|          | 0/1136 [00:00<?, ?it/s]

...


Epoch: 6 
    | Train Loss: 0.019   	| Judge Loss: 0.028
    | Train Accuracy: 0.694 	| Judge Accuracy: 0.836
    | Train F1 Score: 0.605 	| Judge F1 Score: 0.593
Not submitting:
    | F1 Score: 0.593 | Last Submit: 0.643
____________________________________________

Starting epoch 7
    | Train size: 36325
    | Judge size: 4036

  0%|          | 0/1136 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]


Epoch: 7 
    | Train Loss: 0.017   	| Judge Loss: 0.022
    | Train Accuracy: 0.704 	| Judge Accuracy: 0.833
    | Train F1 Score: 0.619 	| Judge F1 Score: 0.658
Not submitting:
    | F1 Score: 0.658 | Last Submit: 0.643
____________________________________________

Starting epoch 8
    | Train size: 36325
    | Judge size: 4036

  0%|          | 0/1136 [00:00<?, ?it/s]